In [ ]:

!pip install roboflow
!pip install ultralytics -q
!pip install gradio -q
!pip install opencv-python matplotlib


from google.colab import drive
import os

drive.mount('/content/drive')


PROJECT_DIR = '/content/drive/MyDrive/LogoRecognition'
os.makedirs(PROJECT_DIR, exist_ok=True)
